# Exporting results for visualisation

We now turn to the exporter facility of {feelpp} which allows to provide the results of {feelpp} solves in a format readable by {paraview} or {ensight}.

In this 2D and 3D example, we


- initiate the {feelpp} environment
- download a mesh description from {uri-github-feelpp}
- build the mesh and partition it
- build a function space $X_h$
- create an element $u$ of $X_h$ which interpolate an expression givn by the end-user
- export to the Ensight Gold format (the default {feelpp} format) of {feelpp}



In [0]:
import feelpp
import sys
app = feelpp.Environment(["myapp"],config=feelpp.localRepository("")) # <1>



1. The {feelpp} environment is initiated with the name of the application and the path to the local repository where the results will be stored.

In [0]:
app.setLogVerbosityLevel(0) # <1>
geofilename=feelpp.download( "github:{repo:feelpp,path:feelpp/quickstart/laplacian/cases/feelpp2d/feelpp2d.geo}", worldComm=app.worldCommPtr() )[0]

mesh = feelpp.load(feelpp.mesh(dim=2,realdim=2), name=geofilename, h=0.1, verbose=1) # <2>

Xh=feelpp.functionSpace(mesh=mesh,space="Pch",order=1) # <3>
P0h=feelpp.functionSpace(mesh=mesh,space="Pdh",order=0) # <4>
u=Xh.element() # <5>
u.on(range=feelpp.elements(mesh), expr=feelpp.expr("sin(2*pi*x)*cos(pi*y):x:y")) # <6>

e = feelpp.exporter(mesh=mesh)  # <7>
e.add("a_scalar", 1.) # <8>
e.add("u",u) # <9>
e.add("pid",feelpp.pid( P0h )) # <10>
e.save()   # <11>



1. The verbosity level is set to 0 (only `VLOG(level)` with `level <= 0` are displayed)
2. The mesh is loaded from the {uri-github-feelpp} repository
3. The function space stem:[X_h] is built from the mesh `mesh` and the space `Pch` and order `1`
4. The function space stem:[P0_h] is built from the mesh `mesh` and the space `Pdh` and order `0`
5. The element `u` is built from the function space `Xh`
6. The element `u` is interpolated with the expression `{sin(2*pi*x)*cos(pi*y)}:x:y`
7. The exporter is built from the mesh `mesh`
8. The scalar `a_scalar` is added to the exporter `e`
9. The element `u` is added to the exporter `e`
10. The element `pid` is added to the exporter `e`
11. The exporter `e` is saved to the disk
*Results*\
```
[loadMesh] Loading mesh in format geo+msh: "/Users/prudhomm/Downloads/feelppdb/downloads/feelpp2d.geo"
[loadMesh] Use default geo desc: /Users/prudhomm/Downloads/feelppdb/downloads/feelpp2d.geo 0.1
```
The results are stored in the directory `feelppdb` and can be visualized with {paraview-website} or {ensight}.

INFO: The file `exports/ensightgold/myapp/myapp.case` can be opened directly in {paraview} or {ensight} to visualize the mesh and the fields.


In [0]:
from xvfbwrapper import Xvfb
vdisplay = Xvfb()
vdisplay.start()
import pyvista as pv
import os
pv.set_jupyter_backend('pythreejs')

def pv_get_mesh(mesh_path):
    reader = pv.get_reader(mesh_path)
    mesh = reader.read()
    return mesh
def pv_plot(mesh, field="u", clim=None, cmap='viridis', cpos='xy', show_scalar_bar=True, show_edges=True):
    mesh.plot(scalars=field, clim=clim, cmap=cmap, cpos=cpos, show_scalar_bar=show_scalar_bar, show_edges=show_edges)

mesh = pv_get_mesh(f"exports/ensightgold/Exporter/Exporter.case")
pv_plot(mesh)
